In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
import sys, os

warnings.filterwarnings('ignore')
random_state = 6
np.random.seed(random_state)

In [2]:
dir_path = '../../pipeline_modules/'
sys.path.append(dir_path)

In [2]:
# data62 = 'seismogram_data_62_new.csv'
# data66 = 'seismogram_data_66_new.csv'
data73 = 'seismogram_data_73_new.csv'
df = pd.read_csv(data73)
print(df.shape)
print(df.columns)

(23460, 73)
Index(['id', 'mfccs_0', 'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5',
       'mfccs_6', 'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11',
       'mfccs_12', 'mfccs_13', 'mfccs_14', 'mfccs_15', 'mfccs_16', 'mfccs_17',
       'mfccs_18', 'mfccs_19', 'mfccs_20', 'mfccs_21', 'mfccs_22', 'mfccs_23',
       'mfccs_24', 'mfccs_25', 'mfccs_26', 'mfccs_27', 'mfccs_28', 'mfccs_29',
       'mfccs_30', 'mfccs_31', 'mfccs_32', 'mfccs_33', 'mfccs_34', 'mfccs_35',
       'mfccs_36', 'mfccs_37', 'mfccs_38', 'mfccs_39', 'chroma_0', 'chroma_1',
       'chroma_2', 'chroma_3', 'chroma_4', 'chroma_5', 'chroma_6', 'chroma_7',
       'chroma_8', 'chroma_9', 'chroma_10', 'chroma_11', 'mel_max', 'mel_mean',
       'power_max', 'power_mean', 'centiroid_mean', 'centiroid_max',
       'centiroid_min', 'max_amplitude', 'mean_amplitude', 'max_psd',
       'welch_max_psd', 'rmse_max', 'rmse_mean', 'moment', 'variation', 'skew',
       'var', 'autocr', 'kurto', 'target'],
      dtype='obje

In [3]:
df.head()

,id,mfccs_0,mfccs_1,mfccs_2,mfccs_3,mfccs_4,mfccs_5,mfccs_6,mfccs_7,mfccs_8,...,welch_max_psd,rmse_max,rmse_mean,moment,variation,skew,var,autocr,kurto,target
0,AE.113A.BHE.M.2013.43.3932,584.034711,67.395719,-1.123569,58.009457,12.714985,26.253530,8.813295,18.570522,28.410674,...,318.812225,8392.133789,4817.334473,0.0,-3.797894e+06,-0.046666,33360.179688,1.681353e+09,-0.228266,1
1,AE.113A.BHN.M.2013.43.3932,585.279273,66.633911,-3.695100,57.115558,9.621800,28.749848,8.592028,16.574689,26.178886,...,247.526642,16092.809570,4819.573242,0.0,3.954760e+06,0.476347,31333.093750,1.579188e+09,1.167933,1
2,AE.113A.BHZ.M.2013.43.3932,582.517055,71.889762,4.812049,57.351785,15.596365,30.629524,11.819666,17.128889,27.543973,...,291.354401,25051.058594,9787.401367,0.0,-7.325927e+06,-0.225913,114514.218750,5.771631e+09,0.067572,1
3,AE.319A.BHE.M.2013.43.3949,671.683429,68.050797,-31.188585,93.386678,11.076502,12.129126,3.464947,28.128156,15.251771,...,248.422974,8914.021484,4352.299805,0.0,7.994568e+05,0.010802,24525.289062,1.236099e+09,0.360999,1
4,AE.319A.BHN.M.2013.43.3949,688.401936,72.066389,-48.951186,98.619771,19.531902,9.367895,-3.468288,21.042504,17.588548,...,256.486481,7046.786621,4361.418457,0.0,8.604528e+05,-0.216832,23690.130859,1.194006e+09,-0.145266,1


### Processing the data

In [4]:
df = shuffle(df, random_state = random_state)
df_train, df_test = train_test_split(df, test_size = 0.20, random_state= random_state)
mms = StandardScaler()
X_train = mms.fit_transform(df_train.drop(['id', 'target', 'moment', 'variation'], axis=1))
Y_train = df_train['target']

# Support Vector machine

In machine learning, support vector machines (SVMs, also support vector networks) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training examples, each marked as belonging to one or the other of two categories, an SVM training algorithm builds a model that assigns new examples to one category or the other, making it a non-probabilistic binary linear classifier (although methods such as Platt scaling exist to use SVM in a probabilistic classification setting). An SVM model is a representation of the examples as points in space, mapped so that the examples of the separate categories are divided by a clear gap that is as wide as possible. New examples are then mapped into that same space and predicted to belong to a category based on which side of the gap they fall.

## Find best parameter using GridSearchCV

Exhaustive search over specified parameter values for an estimator. GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used. The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

In [29]:
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state = random_state)
svm_clss = svm.SVC(class_weight = 'balanced', random_state = random_state) 

param_dist = {'C': np.linspace(0.1, 10, 20), 
              'gamma': np.linspace(0.1, 0.00008, 30)}

# run randomized search
n_iter_search = 20

grid_clf = RandomizedSearchCV(estimator = svm_clss, param_distributions = param_dist, n_iter = 20, cv = cv, n_jobs=-1)

In [30]:
grid_clf.fit(X_train, Y_train.astype(int))

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=6, shuffle=True),
          error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=6, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'C': array([  0.1    ,   0.62105,   1.14211,   1.66316,   2.18421,   2.70526,
         3.22632,   3.74737,   4.26842,   4.78947,   5.31053,   5.83158,
         6.35263,   6.87368,   7.39474,   7.91579,   8.43684,   8.95789,
         9.47895,  10.     ]), 'gamma': array([  1.0000...    1.73076e-02,   1.38621e-02,   1.04166e-02,   6.97103e-03,
         3.52552e-03,   8.00000e-05])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=0)

In [31]:
print(grid_clf.best_params_)
print(grid_clf.best_score_)
print(grid_clf.best_estimator_)

{'gamma': 0.068990344827586203, 'C': 5.8315789473684214}
0.86210571185
SVC(C=5.8315789473684214, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.068990344827586203,
  kernel='rbf', max_iter=-1, probability=False, random_state=6,
  shrinking=True, tol=0.001, verbose=False)


## Use best parameters to make final model 

In [5]:
from sklearn import svm

svm_model = svm.SVC(C=4, cache_size=200, coef0=0.0, class_weight='balanced',
  decision_function_shape=None, degree=3, gamma=0.08, kernel='rbf',
  max_iter=-1, probability=False, random_state = random_state, shrinking=True,
  tol=0.001, verbose=True)

svm_model.fit(X_train, Y_train)

[LibSVM]

SVC(C=4, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.08, kernel='rbf',
  max_iter=-1, probability=False, random_state=6, shrinking=True,
  tol=0.001, verbose=True)

In [6]:
X_test  = mms.fit_transform(df_test.drop(['id', 'target', 'moment', 'variation'], axis=1))
Y_test = df_test['target']

Y_pred = svm_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.8751065643648764

2. Classification report 
              precision    recall  f1-score   support

          0       0.88      0.90      0.89      2594
          1       0.87      0.84      0.86      2098

avg / total       0.88      0.88      0.87      4692
 

3. Confusion matrix 
 [[2337  329]
 [ 257 1769]] 

4. Roc_Auc score 
 0.8748716053643223


In [16]:
classified_df = pd.DataFrame([], columns=['stations', 'original', 'predicted'])
misclassified_df = pd.DataFrame([], columns=['stations', 'original', 'predicted'])

for i, label in enumerate(Y_test):
    result = svm_model.predict(X_test[i, :])
    
    if result[0] == label:    
        classified_df.loc[i] = [df_test.iloc[i, 0], label, result[0]]
    else:
        misclassified_df.loc[i] = [df_test.iloc[i, 0], label, result[0]]

In [22]:
print(classified_df.head())
print('\n----------------------------------------------------------------------\n')
print(misclassified_df.head())

                       stations original predicted
1   US.BOZ.BHE.M.2004.361.11326        0         0
2  NM.BLO.BHZ.M.1998.131.102716        1         1
3   DR.SDD.BHE.M.2010.71.234131        0         0
4   AV.MGOD.BHE.M.2017.246.3375        1         1
5  TA.W18A.BHZ.M.2017.246.34145        1         1

----------------------------------------------------------------------

                        stations original predicted
0   UW.HEBO.BHE.M.2017.246.34025        1         0
49  US.DGMT.BHZ.M.2017.297.11114        0         1
51  N4.V48A.BHN.M.2017.297.11232        0         1
61    CU.TGUH.BH1.M.2016.6.14355        1         0
76  XZ.KIAG.BHE.M.2010.71.233115        0         1


In [21]:
import pickle
data = {'miss': misclassified_df, 'good':classified_df }
output = open('classification_data.pkl', 'wb')
pickle.dump(data, output)
output.close()

### robustness of the model

In [11]:
data_robust = 'seismogram_data_73_new_robust.csv'
df_robust = pd.read_csv(data73)
df_robust = shuffle(df_robust)

In [13]:
df_robust_test = df_robust.drop(['id', 'target', 'moment', 'variation'], axis=1)
df_robust_target = df_robust['target']

X_test  = mms.fit_transform(df_robust_test)
Y_test = df_robust_target

Y_pred = svm_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.9685421994884911

2. Classification report 
              precision    recall  f1-score   support

          0       0.97      0.97      0.97     12962
          1       0.96      0.96      0.96     10498

avg / total       0.97      0.97      0.97     23460
 

3. Confusion matrix 
 [[12594   370]
 [  368 10128]] 

4. Roc_Auc score 
 0.9681992252466491


## Refinement history

<table style="border: 1px solid black;">
  <tr>
    <th>Step</th>
    <th>Method</th> 
    <th>Description</th>
    <th>Number of Features</th>
    <th>Accuracuy</th>
    <th>Pos Recall</th>
    <th>Pos F1 score</th>
    <th>ROC score</th>
  </tr>
  
  <tr>
    <td>Step 0</td>
    <td>Benchmark</td> 
    <td>At this step I used five of supervised algorithoms to find out the best one that gives highest model performance. I have found Support vector machine is the one gives best model performances interms of accuracy, Recall, F-1 and ROC score.</td>
     <td>61</td>
    <td>77%</td> 
    <td>77%</td>
    <td>76%</td>
    <td>77.5%</td> 
  </tr>
  
  <tr>
    <td>Step 1</td>
    <td>Grid search</td> 
    <td>After I select SVM as the model towards final solution, I used scikit-sklearn GridSearcgCV model to find best C and gamma parameters. After exhaustive search with many hours, the best C and gamma are 4.0 and 0.08</td>
     <td>61</td>
    <td>84.5%</td> 
    <td>81%</td>
    <td>81%</td>
    <td>84.5%</td> 
  </tr>
  
    <tr>
    <td>Step 2.1 </td>
    <td>Feature engineering</td> 
    <td>At this stage, I work with feature manipulations. I have tried adding some extra features like amplitude of the signal, mel coefficients etc. BUt the did not improve.</td>
     <td>61-1285</td>
    <td>84.50% - 75.00%</td> 
    <td>81.00% - 74.00%</td>
    <td>81.00% - 78.00%</td>
    <td>84.50% -69.00% </td> 
  </tr>
</table>

## Learning curve to check if more data is required to improve performance

In this step of the project, I will check the learning curve of the support vector machine. A learning curve shows the validation and training score of an estimator for varying numbers of training samples. It is a tool to find out how much we benefit from adding more training data and whether the estimator suffers more from a variance error or a bias error. If both the validation score and the training score converge to a value that is too low with increasing size of the training set, we will not benefit much from more training data. If the training score is much higher than the validation score for the maximum number of training samples, adding more training samples will most likely increase generalization. In the following plot of SVM estimator, we see that the model could benefit from more training examples. We have to add training data to get improved performance.

In [1]:
# %reload_ext autoreload
# %autoreload 2
# import visuals as vs
# vz = vs.vizualization(df)
# vz.check_model_learning(X_train, Y_train, svm_model)

## Bagging of SVM

In this bagging classification ithe above SVM model is the meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator, by introducing randomization into its construction procedure and then making an ensemble out of it.

In [130]:
from sklearn.ensemble import BaggingClassifier
bagging_svm = BaggingClassifier(svm_model, max_samples=0.4, n_estimators=100, n_jobs=-1)
bagging_svm.fit(X_train, Y_train.astype(int))

BaggingClassifier(base_estimator=SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=6, shrinking=True,
  tol=0.001, verbose=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.4, n_estimators=100, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [131]:
Y_pred = bagging_svm.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.7919148936170213

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.82      0.80      0.81      2629
        1.0       0.76      0.78      0.77      2071

avg / total       0.79      0.79      0.79      4700
 

3. Confusion matrix 
 [[2106  455]
 [ 523 1616]] 

4. Roc_Auc score 
 0.7889141232560403
